In [37]:
from sklearn.ensemble import RandomForestRegressor
import pandas as pd

# Read the CSV file into a pandas DataFrame
df = pd.read_csv('/app/data/suumo.csv')

df.head(5)

unique_addresses = df['adress'].unique()
print(f"Total unique addresses: {len(unique_addresses)} of {len(df['adress'])}")

Total unique addresses: 2941 of 135758


In [38]:
numeral_mapping = {
    '１': '一丁目',  # Full-width '1'
    '２': '二丁目',  # Full-width '2'
    '３': '三丁目',  # Full-width '3'
    '４': '四丁目',  # Full-width '4'
    '５': '五丁目',  # Full-width '5'
    '６': '六丁目',  # Full-width '6'
    '７': '七丁目',  # Full-width '7'
    '８': '八丁目',  # Full-width '8'
    '９': '九丁目',  # Full-width '9'
    '１０': '十丁目' # Full-width '10'
}

def convert_address_format(address):
    for arabic, kanji in numeral_mapping.items():
        address = address.replace(arabic, kanji)
    return address

df['converted_address'] = df['adress'].apply(convert_address_format)
print(df[['adress', 'converted_address']])


             adress converted_address
0         東京都目黒区東山１       東京都目黒区東山一丁目
1       東京都世田谷区上野毛１     東京都世田谷区上野毛一丁目
2        東京都世田谷区鎌田１      東京都世田谷区鎌田一丁目
3        東京都世田谷区鎌田１      東京都世田谷区鎌田一丁目
4        東京都世田谷区鎌田１      東京都世田谷区鎌田一丁目
...             ...               ...
135753   東京都杉並区南荻窪４      東京都杉並区南荻窪四丁目
135754    東京都中野区本町２       東京都中野区本町二丁目
135755    東京都練馬区旭町２       東京都練馬区旭町二丁目
135756   東京都中野区江古田１      東京都中野区江古田一丁目
135757    東京都板橋区赤塚２       東京都板橋区赤塚二丁目

[135758 rows x 2 columns]


In [39]:
file_path = '/app/data/geolocation.csv'

jp_geo_df = pd.read_csv(file_path, encoding='shift_jis')
jp_geo_df.head()

,都道府県コード,都道府県名,市区町村コード,市区町村名,大字町丁目コード,大字町丁目名,緯度,経度,原典資料コード,大字・字・丁目区分コード
0,13,東京都,13121,足立区,131210001000,千住橋戸町,35.740193,139.797811,0,1
1,13,東京都,13121,足立区,131210002001,千住緑町一丁目,35.742577,139.792502,0,3
2,13,東京都,13121,足立区,131210002002,千住緑町二丁目,35.744936,139.792197,0,3
3,13,東京都,13121,足立区,131210002003,千住緑町三丁目,35.747166,139.792460,0,3
4,13,東京都,13121,足立区,131210003000,千住河原町,35.743103,139.798787,0,1


In [40]:
df['converted_address'] = df['converted_address'].str.strip()
df['converted_address'] = df['converted_address'].str.replace('ヶ', 'ケ')
jp_geo_df['combined'] = (jp_geo_df['都道府県名'].str.strip() + 
                         jp_geo_df['市区町村名'].str.strip() + 
                         jp_geo_df['大字町丁目名'].str.strip())

# Performing a left join on the address column
result_df = pd.merge(df, jp_geo_df[['combined', '緯度', '経度']], left_on='converted_address', right_on='combined', how='left')
result_df.rename(columns={'緯度': 'latitude', '経度': 'longitude'}, inplace=True)
result_df.drop(["combined"], axis=1, inplace=True)

print(result_df[['converted_address', 'latitude', 'longitude']])

result_df.to_csv('/app/data/suumo_geolocation.csv', index=False)

# Check if any latitude or longitude values are missing
missing_lat = result_df['latitude'].isna().any()
missing_lon = result_df['longitude'].isna().any()

if missing_lat or missing_lon:
    print("There are missing values in latitude or longitude.")
else:
    print("All rows have latitude and longitude.")

# Count missing values
missing_lat_count = result_df['latitude'].isna().sum()
missing_lon_count = result_df['longitude'].isna().sum()

print(f"Missing latitude values: {missing_lat_count}")
print(f"Missing longitude values: {missing_lon_count}")

       converted_address   latitude   longitude
0            東京都目黒区東山一丁目  35.646509  139.692993
1          東京都世田谷区上野毛一丁目  35.611210  139.639622
2           東京都世田谷区鎌田一丁目  35.615537  139.617052
3           東京都世田谷区鎌田一丁目  35.615537  139.617052
4           東京都世田谷区鎌田一丁目  35.615537  139.617052
...                  ...        ...         ...
135753      東京都杉並区南荻窪四丁目  35.701170  139.614911
135754       東京都中野区本町二丁目  35.694769  139.680057
135755       東京都練馬区旭町二丁目  35.771221  139.627184
135756      東京都中野区江古田一丁目  35.725271  139.672652
135757       東京都板橋区赤塚二丁目  35.773922  139.640522

[135758 rows x 3 columns]
All rows have latitude and longitude.
Missing latitude values: 0
Missing longitude values: 0
